<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

# Boilerplate

Much of this can be included as a script within the `~/.jupyter/startup` or `~/.ipython/profile_default/startup` directories, depending on your Jupyter version. Those folders are scraped for python scripts when a notebook starts, and those scripts are executed immediately. Probs don't include import statements there though.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
import os
from collections import namedtuple
import nglview as nv
import mdtraj as md
from copy import copy,deepcopy
import numpy as np
from importlib import reload
from panedr import edr_to_df


import JupyterJoy.brokeh as bk # Requires bokeh 0.12.14 or lower
from JupyterJoy.brokeh import cds_hack
from JupyterJoy.mdpbuild.mdp import MDP20181
from JupyterJoy.funfuncs import make_martini_ndx
import JupyterJoy.pbash

import pandas as pd

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

Loading BokehJS ...

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
%%pbash 
source /store/opt/gromacs2018.1/bin/GMXRC

Lets remind ourselves where we are

In [5]:
nbdir = list(JupyterJoy.__path__)[0]
def cd(target_dir=nbdir):
    """CDs both python and the persistent Bash kernel"""
    os.chdir(target_dir)
    %pbash cd $target_dir
    
class WorkingDirectory():
    """Context manager that cds in and out on enter/exit"""
    def __init__(self, target_dir):
        self.target_dir = target_dir
    def __enter__(self):
        cd()
        cd(self.target_dir)
        return os.getcwd()
    def __exit__(self, *args):
        cd()
    
cd()
%pbash nbdir=$nbdir
%pbash pwd
%pbash ls

/home/joshmitchell/bin/python_packages/JupyterJoy
brokeh.py   demostuff    LICENSE   pbash.py     README.md
demo.ipynb  funfuncs.py  mdpbuild  __pycache__


# Generating a MARTINI system with INSANE

OK so first we need to create our topologies. We'll just follow the INSANE tutorial from http://cgmartini.nl/index.php/tutorials-general-introduction-gmx5/bilayers-gmx5

In [5]:
%pbash mkdir demostuff
with WorkingDirectory('demostuff'):
    %pbash wget http://cgmartini.nl/images/stories/tutorial/2017/bilayer-lipidome-tutorial-GMX5_2017Aug04.tgz
    %pbash tar -xvf bilayer-lipidome-tutorial-GMX5_2017Aug04.tgz

    

mkdir: cannot create directory ‘demostuff’: File exists


: 1

--2018-04-26 17:50:29--  http://cgmartini.nl/images/stories/tutorial/2017/bilayer-lipidome-tutorial-GMX5_2017Aug04.tgz
Resolving cgmartini.nl (cgmartini.nl)... 129.125.177.6
Connecting to cgmartini.nl (cgmartini.nl)|129.125.177.6|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220726467 (211M) [application/x-gzip]
Saving to: 'bilayer-lipidome-tutorial-GMX5_2017Aug04.tgz’

bilayer-lipidome-tu 100%[===================>] 210.50M  8.30MB/s    in 28s     

2018-04-26 17:50:57 (7.60 MB/s) - 'bilayer-lipidome-tutorial-GMX5_2017Aug04.tgz’ saved [220726467/220726467]

bilayer-lipidome-tutorial/
bilayer-lipidome-tutorial/spontaneous-assembly/
bilayer-lipidome-tutorial/spontaneous-assembly/equilbration/
bilayer-lipidome-tutorial/spontaneous-assembly/equilbration/dspe/
bilayer-lipidome-tutorial/spontaneous-assembly/equilbration/dspe/dspe-ext.xtc
bilayer-lipidome-tutorial/spontaneous-assembly/equilbration/dspe/msd.xvg
bilayer-lipidome-tutorial/spontaneous-assembly/equilbrat

bilayer-lipidome-tutorial/refine/angledistr.sh
bilayer-lipidome-tutorial/refine/angle.ndx
bilayer-lipidome-tutorial/refine/system.top
bilayer-lipidome-tutorial/refine/ffG53a6bon.itp
bilayer-lipidome-tutorial/complex-bilayers/
bilayer-lipidome-tutorial/complex-bilayers/raftmix.mdp
bilayer-lipidome-tutorial/complex-bilayers/martini_v2.0_DIPC_01.itp
bilayer-lipidome-tutorial/complex-bilayers/raftmix.edr
bilayer-lipidome-tutorial/complex-bilayers/info.top
bilayer-lipidome-tutorial/complex-bilayers/relax-5.mdp
bilayer-lipidome-tutorial/complex-bilayers/martini_v2.0_ions.itp
bilayer-lipidome-tutorial/complex-bilayers/mdout.mdp
bilayer-lipidome-tutorial/complex-bilayers/minimization.mdp
bilayer-lipidome-tutorial/complex-bilayers/relax-1.mdp
bilayer-lipidome-tutorial/complex-bilayers/martini_v2.0_CHOL_02.itp
bilayer-lipidome-tutorial/complex-bilayers/dppc-dipc-chol.ndx
bilayer-lipidome-tutorial/complex-bilayers/raftmix.gro
bilayer-lipidome-tutorial/complex-bilayers/raftmix.cpt
bilayer-lipidome

Create the initial coordinates and the topology

In [38]:
with WorkingDirectory('demostuff/bilayer-lipidome-tutorial/complex-bilayers'):
    %pb python2 ../scripts/insane.py -l DPPC:4 -l DIPC:3 -l CHOL:3 -salt 0.15 \
                -x 15 -y 10 -z 9 -d 0 -pbc cubic -sol W -o dppc-dipc-chol-insane.gro &> insane.top
    %pb truncate -s -1 dppc-dipc-chol-insane.gro # Remove extra newline
    with open('insane.top', 'r+') as f:
        topstr = '#include "martini_v2.1.itp"\n#include "martini_v2.0_DPPC_01.itp"\n#include "martini_v2.0_DIPC_01.itp"\n#include "martini_v2.0_CHOL_02.itp"\n#include "martini_v2.0_ions.itp"\n\n[ system ]\n; name\nINSANE! Membrane UpperLeaflet>DPPC:DIPC:CHOL=4.0:3.0:3.0 LowerLeaflet>DPPC:DIPC:CHOL=4.0:3.0:3.0\n\n[ molecules ]\n; name  number\n'
        topstr += f.read()
        f.seek(0)
        f.write(topstr)
        f.truncate()
    traj = md.load('dppc-dipc-chol-insane.gro')
    %pb cat insane.top


#include "martini_v2.1.itp"
#include "martini_v2.0_DPPC_01.itp"
#include "martini_v2.0_DIPC_01.itp"
#include "martini_v2.0_CHOL_02.itp"
#include "martini_v2.0_ions.itp"

[ system ]
; name
INSANE! Membrane UpperLeaflet>DPPC:DIPC:CHOL=4.0:3.0:3.0 LowerLeaflet>DPPC:DIPC:CHOL=4.0:3.0:3.0

[ molecules ]
; name  number
; X: 15.000 (19 bins) Y: 10.000 (13 bins) in upper leaflet
; X: 15.000 (19 bins) Y: 10.000 (13 bins) in lower leaflet
; 247 lipids in upper leaflet, 247 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 5312
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 5313 11014
; NDX System 1 11014
; "I mean, the good stuff is just INSANE" --Julia Ormond
DPPC            98
DIPC            74
CHOL            74
DPPC            98
DIPC            74
CHOL            74
W             5580
NA+             61
CL-             61


In [ ]:
view = nv.show_mdtraj(traj)
view.add_spacefill()
view

Run an energy minimisation, then a 30 ns equilibration.

We use JupyterJoy's mdp module for modifying the mdp. It lets you read an MDP into a Python object, modify it in Python, and then write it out as an MDP file again. As it does this, it pretties the file up and switches to consistent, readable, and up-to-date MDP keywords.

In [12]:
with WorkingDirectory('demostuff/bilayer-lipidome-tutorial/complex-bilayers'):
    equil_mdp = MDP20181('martini_md.mdp')
    equil_mdp.energygrps = "non-solvent solvent"
    equil_mdp.tc_grps = "non-solvent solvent"
    equil_mdp.write(tofile="martini_equil.mdp")

Before:
title                    = Martini


; TIMESTEP IN MARTINI 
; Most simulations are numerically stable 
; with dt=40 fs, some (especially rings and polarizable water) require 20-30 fs.
; Note that time steps of 40 fs and larger may create local heating or 
; cooling in your system. Although the use of a heat bath will globally 
; remove this effect, it is advised to check consistency of 
; your results for somewhat smaller time steps in the range 20-30 fs.
; Time steps exceeding 40 fs should not be used; time steps smaller
; than 20 fs are also not required unless specifically stated in the itp file.


integrator               = md
dt                       = 0.03  
nsteps                   = 1000000
nstcomm                  = 100
comm-grps		 = 

nstxout                  = 0
nstvout                  = 0
nstfout                  = 0
nstlog                   = 10000 ; Output frequency for energies to log file 
nstenergy                = 100   ; Output frequency for energies to ener

In [5]:
with WorkingDirectory('demostuff/bilayer-lipidome-tutorial/complex-bilayers'):
    %gmx grompp -f minimization.mdp -c dppc-dipc-chol-insane.gro -p insane.top -o insane-min.tpr
    %gmx mdrun -deffnm insane-min -v
    with open('insane.ndx', 'w') as f:
        traj = md.load('dppc-dipc-chol-insane.gro')
        make_martini_ndx(traj.top, f=f)
    %gmx grompp -f martini_equil.mdp -c insane-min.gro -p insane.top -o insane-md.tpr -n insane.ndx
    %gmx mdrun -deffnm insane-md -v    


                      :-) GROMACS - gmx grompp, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, The GROMACS de

Step=   38, Dmax= 1.1e-01 nm, Epot= -1.44175e+05 Fmax= 7.75352e+03, atom= 4501
Step=   39, Dmax= 1.3e-01 nm, Epot= -1.45469e+05 Fmax= 1.18015e+03, atom= 4501
Step=   40, Dmax= 1.5e-01 nm, Epot= -1.53852e+05 Fmax= 5.42773e+03, atom= 4497
Step=   41, Dmax= 1.8e-01 nm, Epot= -1.56341e+05 Fmax= 5.14576e+02, atom= 4497
Step=   42, Dmax= 2.2e-01 nm, Epot= -1.83585e+05 Fmax= 3.00923e+03, atom= 7839
Step=   43, Dmax= 2.7e-01 nm, Epot= -1.85374e+05 Fmax= 7.79561e+04, atom= 5539
Step=   45, Dmax= 1.6e-01 nm, Epot= -1.88368e+05 Fmax= 2.50619e+04, atom= 5539
Step=   46, Dmax= 1.9e-01 nm, Epot= -1.89051e+05 Fmax= 2.58926e+04, atom= 9391
Step=   48, Dmax= 1.1e-01 nm, Epot= -1.89922e+05 Fmax= 3.68846e+03, atom= 9391
Step=   49, Dmax= 1.4e-01 nm, Epot= -1.91983e+05 Fmax= 8.55973e+03, atom= 6178
Step=   50, Dmax= 1.7e-01 nm, Epot= -1.92526e+05 Fmax= 3.16146e+04, atom= 6750
Step=   52, Dmax= 9.9e-02 nm, Epot= -1.93462e+05 Fmax= 2.41964e+03, atom= 7717
Step=   53, Dmax= 1.2e-01 nm, Epot= -1.95824e+05 Fma

Step=  170, Dmax= 6.8e-02 nm, Epot= -2.46647e+05 Fmax= 8.24061e+02, atom= 6454
Step=  171, Dmax= 8.2e-02 nm, Epot= -2.46796e+05 Fmax= 1.23221e+03, atom= 9527
Step=  172, Dmax= 9.8e-02 nm, Epot= -2.46905e+05 Fmax= 1.32426e+03, atom= 10630
Step=  173, Dmax= 1.2e-01 nm, Epot= -2.47039e+05 Fmax= 7.74940e+02, atom= 9029
Step=  174, Dmax= 1.4e-01 nm, Epot= -2.47246e+05 Fmax= 1.17050e+03, atom= 9329
Step=  175, Dmax= 1.7e-01 nm, Epot= -2.47289e+05 Fmax= 2.36643e+03, atom= 8604
Step=  177, Dmax= 1.0e-01 nm, Epot= -2.47547e+05 Fmax= 7.12964e+02, atom= 5932
Step=  178, Dmax= 1.2e-01 nm, Epot= -2.47764e+05 Fmax= 2.06500e+03, atom= 6930
Step=  179, Dmax= 1.5e-01 nm, Epot= -2.47860e+05 Fmax= 2.47829e+03, atom= 8871
Step=  180, Dmax= 1.8e-01 nm, Epot= -2.47914e+05 Fmax= 3.86953e+03, atom= 7238
Step=  182, Dmax= 1.1e-01 nm, Epot= -2.48035e+05 Fmax= 1.36973e+03, atom= 7899
Step=  183, Dmax= 1.3e-01 nm, Epot= -2.48128e+05 Fmax= 2.69568e+03, atom= 9528
Step=  185, Dmax= 7.6e-02 nm, Epot= -2.48232e+05 Fm

Step=  301, Dmax= 8.7e-02 nm, Epot= -2.57604e+05 Fmax= 2.73671e+03, atom= 6556
Step=  302, Dmax= 1.0e-01 nm, Epot= -2.57679e+05 Fmax= 4.94373e+02, atom= 10867
Step=  303, Dmax= 1.3e-01 nm, Epot= -2.57770e+05 Fmax= 1.35692e+03, atom= 10867
Step=  305, Dmax= 7.6e-02 nm, Epot= -2.57841e+05 Fmax= 3.01674e+02, atom= 6556
Step=  306, Dmax= 9.1e-02 nm, Epot= -2.57989e+05 Fmax= 4.79787e+02, atom= 6556
Step=  308, Dmax= 5.4e-02 nm, Epot= -2.58053e+05 Fmax= 2.08984e+02, atom= 10867
Step=  309, Dmax= 6.5e-02 nm, Epot= -2.58144e+05 Fmax= 1.62350e+03, atom= 6556
Step=  310, Dmax= 7.8e-02 nm, Epot= -2.58223e+05 Fmax= 3.62421e+02, atom= 6556
Step=  311, Dmax= 9.4e-02 nm, Epot= -2.58333e+05 Fmax= 8.49663e+02, atom= 6556
Step=  312, Dmax= 1.1e-01 nm, Epot= -2.58347e+05 Fmax= 1.91927e+03, atom= 10867
Step=  313, Dmax= 1.4e-01 nm, Epot= -2.58380e+05 Fmax= 1.56861e+03, atom= 9150
Step=  314, Dmax= 1.6e-01 nm, Epot= -2.58461e+05 Fmax= 3.23895e+02, atom= 9886
Step=  316, Dmax= 9.7e-02 nm, Epot= -2.58549e+05

   2147   2148   35.2    0.2741   0.2756      0.2719
Step=  389, Dmax= 1.2e-01 nm, Epot= -2.61949e+05 Fmax= 1.10013e+03, atom= 4369
Step=  391, Dmax= 7.0e-02 nm, Epot= -2.62023e+05 Fmax= 5.02540e+02, atom= 5147
Step=  393, Dmax= 4.2e-02 nm, Epot= -2.62068e+05 Fmax= 1.35329e+02, atom= 5147
Step=  395, Dmax= 2.5e-02 nm, Epot= -2.62127e+05 Fmax= 4.18490e+02, atom= 8172
Step=  396, Dmax= 3.0e-02 nm, Epot= -2.62158e+05 Fmax= 2.10143e+02, atom= 8172
Step=  397, Dmax= 3.6e-02 nm, Epot= -2.62206e+05 Fmax= 5.05702e+02, atom= 8172
Step=  398, Dmax= 4.3e-02 nm, Epot= -2.62236e+05 Fmax= 3.51083e+02, atom= 8172
Step=  399, Dmax= 5.2e-02 nm, Epot= -2.62267e+05 Fmax= 7.27407e+02, atom= 8172
Step=  400, Dmax= 6.3e-02 nm, Epot= -2.62298e+05 Fmax= 5.18050e+02, atom= 8172
Step=  401, Dmax= 7.5e-02 nm, Epot= -2.62311e+05 Fmax= 1.15188e+03, atom= 8172
Step=  402, Dmax= 9.0e-02 nm, Epot= -2.62346e+05 Fmax= 6.81652e+02, atom= 8172
Step=  404, Dmax= 5.4e-02 nm, Epot= -2.62392e+05 Fmax= 2.46826e+02, atom= 2292

Step=  520, Dmax= 6.2e-02 nm, Epot= -2.65289e+05 Fmax= 5.36823e+02, atom= 734
Step=  522, Dmax= 3.7e-02 nm, Epot= -2.65308e+05 Fmax= 4.03368e+02, atom= 8888
Step=  523, Dmax= 4.5e-02 nm, Epot= -2.65328e+05 Fmax= 4.19896e+02, atom= 8888
Step=  524, Dmax= 5.4e-02 nm, Epot= -2.65348e+05 Fmax= 5.75648e+02, atom= 8888
Step=  525, Dmax= 6.4e-02 nm, Epot= -2.65367e+05 Fmax= 5.61450e+02, atom= 8888
Step=  526, Dmax= 7.7e-02 nm, Epot= -2.65369e+05 Fmax= 1.09394e+03, atom= 8888
Step=  527, Dmax= 9.3e-02 nm, Epot= -2.65398e+05 Fmax= 6.39757e+02, atom= 6811
Step=  529, Dmax= 5.6e-02 nm, Epot= -2.65435e+05 Fmax= 3.58349e+02, atom= 7337
Step=  530, Dmax= 6.7e-02 nm, Epot= -2.65472e+05 Fmax= 3.38152e+02, atom= 6811
Step=  532, Dmax= 4.0e-02 nm, Epot= -2.65492e+05 Fmax= 6.47990e+02, atom= 8888
Step=  533, Dmax= 4.8e-02 nm, Epot= -2.65521e+05 Fmax= 2.99526e+02, atom= 8888
Step=  534, Dmax= 5.8e-02 nm, Epot= -2.65551e+05 Fmax= 6.80469e+02, atom= 8888
Step=  535, Dmax= 6.9e-02 nm, Epot= -2.65569e+05 Fmax

bonds that rotated more than 30 degrees:
 atom 1 atom 2  angle  previous, current, constraint length
   5147   5148   31.5    0.2717   0.2737      0.2719
Step=  649, Dmax= 5.5e-02 nm, Epot= -2.67751e+05 Fmax= 2.40641e+02, atom= 4940
Step=  651, Dmax= 3.3e-02 nm, Epot= -2.67784e+05 Fmax= 1.86110e+02, atom= 8888
Step=  652, Dmax= 4.0e-02 nm, Epot= -2.67803e+05 Fmax= 6.45564e+02, atom= 8888
Step=  653, Dmax= 4.8e-02 nm, Epot= -2.67832e+05 Fmax= 2.85771e+02, atom= 8888
Step=  654, Dmax= 5.7e-02 nm, Epot= -2.67841e+05 Fmax= 1.02497e+03, atom= 8888
Step=  655, Dmax= 6.9e-02 nm, Epot= -2.67870e+05 Fmax= 3.90607e+02, atom= 8888
Step=  657, Dmax= 4.1e-02 nm, Epot= -2.67893e+05 Fmax= 2.25312e+02, atom= 7348
Step=  658, Dmax= 4.9e-02 nm, Epot= -2.67910e+05 Fmax= 9.98988e+02, atom= 8888
Step=  659, Dmax= 5.9e-02 nm, Epot= -2.67934e+05 Fmax= 4.08562e+02, atom= 8888
Step=  660, Dmax= 7.1e-02 nm, Epot= -2.67946e+05 Fmax= 1.04138e+03, atom= 8888
Step=  661, Dmax= 8.5e-02 nm, Epot= -2.67961e+05 Fmax= 4

Step=  773, Dmax= 4.7e-02 nm, Epot= -2.69586e+05 Fmax= 8.18997e+02, atom= 7564
Step=  774, Dmax= 5.7e-02 nm, Epot= -2.69598e+05 Fmax= 4.73625e+02, atom= 7564
Step=  776, Dmax= 3.4e-02 nm, Epot= -2.69618e+05 Fmax= 1.96971e+02, atom= 7653
Step=  777, Dmax= 4.1e-02 nm, Epot= -2.69647e+05 Fmax= 4.35497e+02, atom= 7653
Step=  778, Dmax= 4.9e-02 nm, Epot= -2.69651e+05 Fmax= 5.72687e+02, atom= 7564
Step=  780, Dmax= 2.9e-02 nm, Epot= -2.69668e+05 Fmax= 1.68098e+02, atom= 5794
Step=  781, Dmax= 3.5e-02 nm, Epot= -2.69700e+05 Fmax= 4.41549e+02, atom= 5794
Step=  782, Dmax= 4.2e-02 nm, Epot= -2.69704e+05 Fmax= 4.12017e+02, atom= 7564
Step=  783, Dmax= 5.1e-02 nm, Epot= -2.69712e+05 Fmax= 6.35699e+02, atom= 7564
Step=  784, Dmax= 6.1e-02 nm, Epot= -2.69714e+05 Fmax= 6.24048e+02, atom= 7564
Step=  786, Dmax= 3.7e-02 nm, Epot= -2.69741e+05 Fmax= 1.76867e+02, atom= 7653
Step=  787, Dmax= 4.4e-02 nm, Epot= -2.69778e+05 Fmax= 4.53053e+02, atom= 7653
Step=  788, Dmax= 5.3e-02 nm, Epot= -2.69778e+05 Fma

Step=  904, Dmax= 6.0e-02 nm, Epot= -2.71085e+05 Fmax= 1.16550e+03, atom= 7564
Step=  905, Dmax= 7.3e-02 nm, Epot= -2.71113e+05 Fmax= 5.44786e+02, atom= 5794
Step=  907, Dmax= 4.4e-02 nm, Epot= -2.71138e+05 Fmax= 1.43914e+02, atom= 7653
Step=  908, Dmax= 5.2e-02 nm, Epot= -2.71146e+05 Fmax= 8.24458e+02, atom= 7653
Step=  909, Dmax= 6.3e-02 nm, Epot= -2.71155e+05 Fmax= 1.20575e+03, atom= 7564
Step=  910, Dmax= 7.5e-02 nm, Epot= -2.71169e+05 Fmax= 5.79060e+02, atom= 5794
Step=  912, Dmax= 4.5e-02 nm, Epot= -2.71194e+05 Fmax= 1.34452e+02, atom= 7653
Step=  913, Dmax= 5.4e-02 nm, Epot= -2.71199e+05 Fmax= 7.99564e+02, atom= 7653
Step=  915, Dmax= 3.2e-02 nm, Epot= -2.71242e+05 Fmax= 2.09434e+02, atom= 7564
Step=  916, Dmax= 3.9e-02 nm, Epot= -2.71260e+05 Fmax= 2.83598e+02, atom= 7564
Step=  917, Dmax= 4.7e-02 nm, Epot= -2.71268e+05 Fmax= 5.17902e+02, atom= 7564
Step=  919, Dmax= 2.8e-02 nm, Epot= -2.71280e+05 Fmax= 9.48786e+01, atom= 5794
Step=  920, Dmax= 3.4e-02 nm, Epot= -2.71312e+05 Fma

Excluding 1 bonded neighbours molecule type 'DIPC'
Excluding 1 bonded neighbours molecule type 'CHOL'
Excluding 1 bonded neighbours molecule type 'DPPC'
Excluding 1 bonded neighbours molecule type 'DIPC'
Excluding 1 bonded neighbours molecule type 'CHOL'
Excluding 1 bonded neighbours molecule type 'W'
Excluding 1 bonded neighbours molecule type 'NA+'
Excluding 1 bonded neighbours molecule type 'CL-'
Velocities were taken from a Maxwell distribution at 320 K
Cleaning up constraints and constant bonded interactions with virtual sites
Removing all charge groups because cutoff-scheme=Verlet
Number of degrees of freedom in T-Coupling group non-solvent is 13862.66
Number of degrees of freedom in T-Coupling group solvent is 17074.34
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 323 K

NOTE 1 [file martini_equil.mdp]:
  There are 296 non-linear virtual site constructions. Their contribution
  to the energy error is approximated. In most cases this does not affect
  the error 

# Analysis

We can visualise the whole trajectory right here!

In [6]:
with WorkingDirectory('demostuff/bilayer-lipidome-tutorial/complex-bilayers'):
    equil_traj = md.load('insane-md.xtc', top='insane-md.gro')
view = nv.show_mdtraj(equil_traj)
solvent = 'W OR NA+ OR CL-'
view.add_spacefill(selection='W OR NA+ OR CL-', radius_type='size', radius=3, opacity=0.5)
view.add_spacefill(selection='DPP'.format(solvent), radius_type='size', radius=3, color='purple', roughness=1)
view.add_spacefill(selection='DIP'.format(solvent), radius_type='size', radius=3, color='green', roughness=1)
view.add_spacefill(selection='CHO'.format(solvent), radius_type='size', radius=3, color='yellow', roughness=1)
view.center()
view

NGLWidget(count=1001)

We can even read the EDR file and view it prettily.

In [7]:
with WorkingDirectory('demostuff/bilayer-lipidome-tutorial/complex-bilayers'):
    edr_df = edr_to_df("insane-md.edr", verbose=True)
    
edr_df_skip = edr_df[:][::10]
bk.plot_vs_time(["Potential", "Kinetic En.", 'Total Energy'], edr_df_skip, equil_traj, ploty_label='Energy')

FrameLinkedPlot(children=(BokehWrapper(Figure(id='9bb7e3d7-ea61-48bf-ab85-62aaf3e87bb2', ...), ...), Button(description='Next selected frame', layout=Layout(height='30px'), style=ButtonStyle(), tooltip='Set NGLView frame to the next selected point'), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='600px', width='500px'))), layout=Layout(align_items='center', display='flex', flex_flow='column wrap', height='650px', width='100%'))

Unfortunately the default viewing settings aren't great... but since this is python, we can fix this with a function!

In [8]:
def plot_vs_time_insane(*args, **kwargs):
    linkwidg = bk.plot_vs_time(*args, **kwargs)
    for view in linkwidg.views:
        view.clear_representations()
        solvent = 'W OR NA+ OR CL-'
        view.add_spacefill(selection=solvent, radius_type='size', radius=3, opacity=0.5)
        view.add_spacefill(selection='DPP'.format(solvent), radius_type='size', radius=3, color='purple', roughness=1)
        view.add_spacefill(selection='DIP'.format(solvent), radius_type='size', radius=3, color='green', roughness=1)
        view.add_spacefill(selection='CHO'.format(solvent), radius_type='size', radius=3, color='yellow', roughness=1)
        view.center()
    return linkwidg

plot_vs_time_insane(["Potential", "Kinetic En.", 'Total Energy'], edr_df_skip, equil_traj, ploty_label='Energy')

FrameLinkedPlot(children=(BokehWrapper(Figure(id='ae658c7c-6791-42a7-bd81-87549d3e0024', ...), ...), Button(description='Next selected frame', layout=Layout(height='30px'), style=ButtonStyle(), tooltip='Set NGLView frame to the next selected point'), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='600px', width='500px'))), layout=Layout(align_items='center', display='flex', flex_flow='column wrap', height='650px', width='100%'))

We can calculate and visualise arbitrary properties on the fly:

In [9]:
box_x = edr_df_skip['Box-X']
box_y = edr_df_skip['Box-Y']
area = box_x * box_y
lipid_resnames = set(['DPPC', 'DIPC', 'CHOL'])
n_lipids = len([res for res in equil_traj.top.residues if res.name in lipid_resnames])
area_per_lipid = area/n_lipids
lw = plot_vs_time_insane(0, [area_per_lipid], equil_traj, ploty_label='Area per lipid')
lw

FrameLinkedPlot(children=(BokehWrapper(Figure(id='60bcbda7-14a9-4e06-8723-4eb5ac8df3f3', ...), ...), Button(description='Next selected frame', layout=Layout(height='30px'), style=ButtonStyle(), tooltip='Set NGLView frame to the next selected point'), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='600px', width='500px'))), layout=Layout(align_items='center', display='flex', flex_flow='column wrap', height='650px', width='100%'))

We can actually do some really cool stuff with this machinery. Let's break the trajectory into the three lipid types, and calculate some rando property we can compare between them.

In [15]:
top = equil_traj.top
trajs = {}
bilayer_widths = {}
lipid_resnames = set(['DPPC', 'DIPC', 'CHOL'])
# lipid_resnames.add(' '.join(lipid_resnames))

for name in lipid_resnames:
    idcs = top.select('resname ' + name)
    traj = equil_traj.atom_slice(idcs)
    trajs[name] = traj
    
    bilayer_widths[name] = np.max(traj.xyz[:,:,2], axis=1) - np.min(traj.xyz[:,:,2], axis=1)
    

And now we visualise by specifying the indices, and the variable and trajectory dicts.

In [16]:
lw = bk.plot_vs_time(lipid_resnames, bilayer_widths, trajs, ploty_label='Bilayer height')
for view in lw.views:
    view.clear_representations()
    view.add_spacefill(radius_type='size', radius=3, color=view._colour, roughness=1)
lw

FrameLinkedPlot(children=(BokehWrapper(Figure(id='b3c2a392-32e1-4c26-ae00-8727e2966f65', ...), ...), Button(description='Next selected frame', layout=Layout(height='30px'), style=ButtonStyle(), tooltip='Set NGLView frame to the next selected point'), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='300px', width='250px')), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='300px', width='250px')), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='300px', width='250px'))), layout=Layout(align_items='center', display='flex', flex_flow='column wrap', height='650px', width='100%'))

Brokeh's `FrameLinkedWidget` makes it possible to plot arbitrary data against arbitrary trajectories. `plot_vs_time` is a  helper function supposed to make things easier. Whether it achieves that is completely up for debate, and of course it can't be used if you want to use something other than time on the x-axis Here's how to produce the above chart by hand.

In [31]:
# We already have dictionaries with matching keys for the trajectories and data, so lets get the same for the times
times = {k:t.time for k,t in trajs.items()}

# FrameLinkedPlots work by matching the order of colvars with that of trajs
# so we need a defined order
names = list(lipid_resnames)

# Now we just create an array that holds all that data
# it should have shape (ntrajs, nframes, whatever)
# We have two pieces of data for each of 3 trajectories that have 1001 frames each, 
# so we'll have (3, 1001, 2)
# We have our ordering of the trajectories, so we can do the first axis as a list comp
# the second two axes are just the two existing data structures stacked together
data = np.asarray([np.stack([times[k], bilayer_widths[k]], axis=-1) for k in names])

# We also need a list of trajectories in the same order
traj_to_plot = [trajs[k] for k in names]

# Last trick: we need to tell FrameLinkedPlot which 
# parts of the array to plot. time is in data[:,:,0],
# and we want it on the x axis. Width is in data[:,:,1].
# This lets us give FrameLinkedPlot a huge amount of data,
# or even a dataframe, and then change what we plot in the cell easily
idx_on_x = 0
idx_on_y = 1

plotx_label = "Time (ps)"
ploty_label = "Bilayer height (A)"

linkwidg = bk.FrameLinkedPlot(data, traj_to_plot, 
                              plotx_label=plotx_label, 
                              ploty_label=ploty_label, 
                              stride=1, 
                              title='Bilayer width against time',
                              plotx=idx_on_x,
                              ploty=idx_on_y)

for view in linkwidg.views:
    view.clear_representations()
    view.add_spacefill(radius_type='size', radius=3, color=view._colour, roughness=1)

linkwidg

FrameLinkedPlot(children=(BokehWrapper(Figure(id='03ab0f94-360b-4eb4-a6de-9a68eea5e5cd', ...), ...), Button(description='Next selected frame', layout=Layout(height='30px'), style=ButtonStyle(), tooltip='Set NGLView frame to the next selected point'), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='300px', width='250px')), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='300px', width='250px')), SeleNGLWidget(count=1001, layout=Layout(flex='0 0 auto', height='300px', width='250px'))), layout=Layout(align_items='center', display='flex', flex_flow='column wrap', height='650px', width='100%'))